In [ ]:
import sys
!{sys.executable} -m pip install matplotlib

https://www.tensorflow.org/tutorials/keras/text_classification

In [2]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

In [3]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)

2021-12-21 20:32:09.005985: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Failed precondition: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling imdb_reviews-test.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-unsupervised.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2021-12-21 20:33:03.950128: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 20:33:04.040594: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 20:33:04.041294: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 20:33:04.043567: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [5]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


2021-12-21 20:33:14.851201: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-21 20:33:14.897499: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [6]:
x_train = []
x_test = []
y_train = []
y_test = []

randomized_train_dataset = dataset["train"]

iterator = iter(randomized_train_dataset)
while(1):
  val = iterator.get_next_as_optional()  
  if(val.has_value().numpy() == False):
    break
  data, lbl = val.get_value()
  x_train.append(str(data.numpy().decode("utf-8")))
  y_train.append(lbl.numpy())


randomized_test_dataset = dataset["test"]

iterator = iter(randomized_test_dataset)

while(1):
  val = iterator.get_next_as_optional()  
  if(val.has_value().numpy() == False):
    break
  data, lbl = val.get_value()
  x_test.append(str(data.numpy().decode("utf-8")))
  y_test.append(lbl.numpy())

In [7]:
random_y_train = np.random.choice(2, len(x_train))

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train,random_y_train))

In [9]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [10]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [11]:
VOCAB_SIZE = 5000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [12]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [13]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [14]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10


2021-12-21 20:34:35.951704: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


391/391 [==============================] - 73s 151ms/step - loss: 0.6932 - accuracy: 0.5006 - val_loss: 0.6933 - val_accuracy: 0.4917
Epoch 2/10
391/391 [==============================] - 54s 139ms/step - loss: 0.6930 - accuracy: 0.5006 - val_loss: 0.6933 - val_accuracy: 0.4917
Epoch 3/10
391/391 [==============================] - 55s 140ms/step - loss: 0.6919 - accuracy: 0.5006 - val_loss: 0.6940 - val_accuracy: 0.4917
Epoch 4/10
391/391 [==============================] - 55s 140ms/step - loss: 0.6811 - accuracy: 0.5140 - val_loss: 0.7108 - val_accuracy: 0.4964
Epoch 5/10
391/391 [==============================] - 55s 141ms/step - loss: 0.6524 - accuracy: 0.5838 - val_loss: 0.7392 - val_accuracy: 0.4802
Epoch 6/10
391/391 [==============================] - 54s 139ms/step - loss: 0.6214 - accuracy: 0.6360 - val_loss: 0.7724 - val_accuracy: 0.4818
Epoch 7/10
391/391 [==============================] - 55s 140ms/step - loss: 0.5907 - accuracy: 0.6748 - val_loss: 0.7987 - val_accuracy: 0.4

In [15]:
history = model.fit(train_dataset, epochs=5,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/5
391/391 [==============================] - 55s 140ms/step - loss: 0.4808 - accuracy: 0.7779 - val_loss: 0.9950 - val_accuracy: 0.4839
Epoch 2/5
391/391 [==============================] - 55s 142ms/step - loss: 0.4599 - accuracy: 0.7955 - val_loss: 1.0020 - val_accuracy: 0.4792
Epoch 3/5
391/391 [==============================] - 55s 141ms/step - loss: 0.4366 - accuracy: 0.8102 - val_loss: 1.1175 - val_accuracy: 0.4750
Epoch 4/5
391/391 [==============================] - 55s 141ms/step - loss: 0.4154 - accuracy: 0.8265 - val_loss: 1.1726 - val_accuracy: 0.4724
Epoch 5/5
391/391 [==============================] - 55s 141ms/step - loss: 0.3916 - accuracy: 0.8431 - val_loss: 1.2598 - val_accuracy: 0.4714


In [17]:
history = model.fit(train_dataset, epochs=20,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/20
391/391 [==============================] - 55s 140ms/step - loss: 0.3638 - accuracy: 0.8563 - val_loss: 1.3870 - val_accuracy: 0.4812
Epoch 2/20
391/391 [==============================] - 55s 141ms/step - loss: 0.3387 - accuracy: 0.8704 - val_loss: 1.4317 - val_accuracy: 0.4781
Epoch 3/20
391/391 [==============================] - 55s 141ms/step - loss: 0.3193 - accuracy: 0.8808 - val_loss: 1.6038 - val_accuracy: 0.4776
Epoch 4/20
391/391 [==============================] - 55s 141ms/step - loss: 0.3024 - accuracy: 0.8867 - val_loss: 1.6494 - val_accuracy: 0.4760
Epoch 5/20
391/391 [==============================] - 55s 141ms/step - loss: 0.2825 - accuracy: 0.8956 - val_loss: 1.7812 - val_accuracy: 0.4823
Epoch 6/20
391/391 [==============================] - 55s 141ms/step - loss: 0.2675 - accuracy: 0.9012 - val_loss: 1.8328 - val_accuracy: 0.4766
Epoch 7/20
391/391 [==============================] - 54s 139ms/step - loss: 0.2527 - accuracy: 0.9070 - val_loss: 1.9531 - val_ac

KeyboardInterrupt: 